# <center>Searching in scientific papers</center>
##  <center>Patrik Kovács</center>
###  <center>Subject: Information retrieval </center>

#Preprocessing

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Used libraries

In [3]:
%pip install ijson==3.1.2

     |████████████████████████████████| 133kB 2.8MB/s 


In [4]:
import json
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import nltk
import json
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import ijson
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#downloaded = drive.CreateFile({'id':'1JRSqi4HDhIHXtOBo4P6M30qrX2o-IS2z'}) 
#downloaded.GetContentFile('arxiv-metadata-oai-snapshot.jsonb')

In [ ]:
downloaded = drive.CreateFile({'id':'1-6gFZm7ZAKlMVO_K3qpdJpU7TQKYZPZX'}) 
downloaded.GetContentFile('papers.json')

In [ ]:
#with open('papers.json') as f:
  #data = json.load(f)

In [ ]:
#with open('papers_medium.json', 'w') as json_file:
  #json.dump(data[:1000], json_file)

In [ ]:
#df=pd.read_json('arxiv-metadata-oai-snapshot.json',lines = True)

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#df=pd.read_json('arxiv-metadata-oai-snapshot.json',lines = True)

In [5]:
import ijson
import json
import re
import numpy as np
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

docs = []
count = 0
remove = False
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("module %s loaded" % module_url)


def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)


def clean_text(text):
    # remove backslash-apostrophe
    text = re.sub("\'", "", text)
    # remove everything except alphabets and numbers
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    # remove whitespaces
    text = ' '.join(text.split())
    # convert text to lowercase
    text = text.lower()

    return text


def format_sentence_encoder_result_to_array(arr):
    return str(np.array(arr[0]).tolist())


def append_to_json(_dict, path):
    with open(path, 'ab+') as f:
        f.seek(0, 2)  # Go to the end of file
        if f.tell() == 0:  # Check if file is empty
            f.write(json.dumps(_dict).encode())  # If empty, write an array
        else:
            pos = f.seek(-1, 2)
            f.truncate()  # Remove the last character, open the array
            f.write(' , '.encode())  # Write the separator
            # Write after from [ character
            f.write(json.dumps(_dict).encode()[1:])


def get_page_number(str):
    try:
        found = re.search('([0-9]+) +[pP]ages?', str).group(1)
    except AttributeError:
        found = 'No data'
    return found


def get_figure_number(str):
    try:
        found = re.search('([0-9]+) +[fF]igures?', str).group(1)
    except AttributeError:
        found = 'No data'
    return found


def categories_to_list_of_strings(categories):
    return str(categories.split(' '))


def get_version_date(versions):
    versions = eval(versions)
    return str(versions[-1]['created'])


def get_version_number(versions):
    versions = eval(versions)
    return str(versions[-1]['version'])


def get_list_of_authors(authors):
    authors = eval(authors)
    res = list(map(' '.join, authors))
    return str(list(map(str.strip, res)))


with open('papers.json', 'rb') as data:
    for obj in ijson.items(data, 'item'):
        # filter out not accepted papers
        if(bool(re.search('[pP]aper.*[wW]ithdrawn|^[Ww]ithdrawn', obj['abstract']))):
            continue
         # vectorize abstract
        abstract = clean_text(obj['abstract'])
        abstract = remove_stopwords(abstract)
        abstract = model([abstract])           # google sentence encoder
        abstract = format_sentence_encoder_result_to_array(abstract)
        pages = get_page_number(obj['comments'])
        figures = get_figure_number(obj['comments'])
        categories = categories_to_list_of_strings(obj['categories'])
        latest_version_date = get_version_date(obj['versions'])
        latest_version = get_version_number(obj['versions'])
        list_of_authors = get_list_of_authors(obj['authors_parsed'])

        body = {
            "id": obj['id'],
            "submitter": obj['submitter'],
            "title": obj['title'],
            "journal_ref": obj['journal-ref'],
            "doi": obj['doi'],
            "report_no": obj['report-no'],
            "categories": categories,
            "license": obj['license'],
            "abstract": abstract,
            "update_date": obj['update_date'],
            "pages": pages,
            "figures": figures,
            "latest_version_date": latest_version_date,
            "latest_version": latest_version,
            "list_of_authors": list_of_authors
        }
        docs.append(body)
        count += 1
        if count % 100 == 0:
            append_to_json(docs, 'res.json')
            docs = []
            print("saved {} documents.".format(count))

if docs:
    append_to_json(docs, 'res.json')
    docs = []
    print("saved {} documents.".format(count))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
module https://tfhub.dev/google/universal-sentence-encoder/4 loaded
saved 100 documents.
saved 200 documents.
saved 300 documents.
saved 400 documents.
saved 500 documents.
saved 600 documents.
saved 700 documents.
saved 800 documents.
saved 900 documents.
saved 997 documents.
